# Fluid Dynamics-Based Risk Management Strategies

## Table of Contents
1. [Introduction to Fluid Risk Models](#introduction)
2. [Turbulence-Based Risk Metrics](#turbulence-risk)
3. [Vorticity Risk Assessment](#vorticity-risk)
4. [Energy Dissipation and Drawdown](#energy-dissipation)
5. [Dynamic Position Sizing](#position-sizing)
6. [Portfolio Flow Correlation](#portfolio-flow)
7. [Stress Testing with Fluid Scenarios](#stress-testing)
8. [Risk-Adjusted Performance Metrics](#performance-metrics)

## Introduction

Traditional risk management relies on statistical measures like volatility and VaR. Fluid dynamics provides a richer framework for understanding market risk through:

- **Turbulence intensity**: Measure of market chaos and unpredictability
- **Vorticity patterns**: Early warning signals for trend reversals
- **Energy dissipation**: Rate of capital erosion during adverse conditions
- **Flow coherence**: Correlation structure across assets and timeframes
- **Regime stability**: Persistence and transition probabilities

### Key Advantages:
- **Forward-looking**: Anticipates risk before it materializes
- **Multi-dimensional**: Captures complex risk interactions
- **Dynamic**: Adapts to changing market conditions
- **Regime-aware**: Risk metrics adjust based on market state

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats, signal
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import LedoitWolf
import seaborn as sns
from matplotlib.patches import Ellipse
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("Set2")

print("Fluid Dynamics Risk Management Environment Initialized")
print("Advanced risk models ready for deployment")

## Turbulence-Based Risk Metrics

### Turbulent Kinetic Energy (TKE)

TKE measures the intensity of market turbulence:
$$TKE = \frac{1}{2}\langle u'^2 + v'^2 \rangle$$

Where $u'$ and $v'$ are velocity fluctuations from the mean flow.

### Reynolds Stress Tensor

Captures correlations between velocity components:
$$R_{ij} = \langle u'_i u'_j \rangle$$

### Turbulence Intensity

Normalized measure of turbulence:
$$I = \frac{\sqrt{TKE}}{|\vec{U}|}$$

In [ ]:
class FluidRiskManager:
    """
    Comprehensive risk management using fluid dynamics principles
    """
    
    def __init__(self, lookback_window=252):
        self.lookback_window = lookback_window
        self.risk_metrics = {}
        self.position_limits = {}
        self.stress_scenarios = {}
        
    def generate_market_data(self, n_assets=5, n_periods=1000):
        """
        Generate synthetic multi-asset market data with regime changes
        """
        np.random.seed(42)
        
        # Create correlation matrix
        correlation_base = np.random.uniform(0.1, 0.7, (n_assets, n_assets))
        correlation_matrix = (correlation_base + correlation_base.T) / 2
        np.fill_diagonal(correlation_matrix, 1.0)
        
        # Generate different market regimes
        regime_changes = [0, 250, 500, 750, 1000]
        regimes = ['normal', 'volatile', 'trending', 'crisis']
        
        returns_data = []
        prices_data = []
        regime_labels = []
        
        for i in range(len(regime_changes)-1):
            start_idx = regime_changes[i]
            end_idx = regime_changes[i+1]
            period_length = end_idx - start_idx
            
            regime = regimes[i % len(regimes)]
            
            # Regime-specific parameters
            if regime == 'normal':
                volatility = np.random.uniform(0.15, 0.25, n_assets)
                mean_return = np.random.uniform(-0.001, 0.002, n_assets)
            elif regime == 'volatile':
                volatility = np.random.uniform(0.4, 0.8, n_assets)
                mean_return = np.random.uniform(-0.002, 0.001, n_assets)
            elif regime == 'trending':
                volatility = np.random.uniform(0.1, 0.2, n_assets)
                mean_return = np.random.uniform(0.001, 0.005, n_assets)
            else:  # crisis
                volatility = np.random.uniform(0.6, 1.2, n_assets)
                mean_return = np.random.uniform(-0.01, -0.002, n_assets)
                # Increase correlations during crisis
                correlation_matrix = np.maximum(correlation_matrix, 0.8)
                np.fill_diagonal(correlation_matrix, 1.0)
            
            # Generate correlated returns
            L = np.linalg.cholesky(correlation_matrix)
            uncorr_returns = np.random.normal(0, 1, (period_length, n_assets))
            corr_returns = uncorr_returns @ L.T
            
            # Scale by volatility and add mean
            scaled_returns = corr_returns * volatility + mean_return
            
            returns_data.append(scaled_returns)
            regime_labels.extend([regime] * period_length)
        
        # Combine all periods
        returns = np.vstack(returns_data)
        
        # Generate prices from returns
        prices = np.zeros_like(returns)
        prices[0] = 100.0  # Initial price
        for t in range(1, len(returns)):
            prices[t] = prices[t-1] * (1 + returns[t])
        
        # Create timestamps
        timestamps = pd.date_range('2023-01-01', periods=n_periods, freq='D')
        
        return {
            'returns': pd.DataFrame(returns, index=timestamps, 
                                  columns=[f'Asset_{i+1}' for i in range(n_assets)]),
            'prices': pd.DataFrame(prices, index=timestamps,
                                 columns=[f'Asset_{i+1}' for i in range(n_assets)]),
            'regimes': pd.Series(regime_labels, index=timestamps, name='regime')
        }
    
    def compute_turbulence_metrics(self, returns_df):
        """
        Compute fluid dynamics-based turbulence metrics
        """
        returns = returns_df.values
        n_periods, n_assets = returns.shape
        
        # Compute velocity components (price momentum)
        velocity_x = np.gradient(returns, axis=0)  # Time derivative
        velocity_y = np.gradient(returns, axis=1)  # Cross-asset derivative
        
        # Mean flow
        mean_vx = np.mean(velocity_x, axis=0)
        mean_vy = np.mean(velocity_y, axis=0)
        
        # Velocity fluctuations
        vx_prime = velocity_x - mean_vx
        vy_prime = velocity_y - mean_vy
        
        # Turbulent Kinetic Energy
        tke = 0.5 * (vx_prime**2 + vy_prime**2)
        
        # Reynolds stress tensor components
        reynolds_uu = np.mean(vx_prime**2, axis=0)
        reynolds_vv = np.mean(vy_prime**2, axis=0)
        reynolds_uv = np.mean(vx_prime * vy_prime, axis=0)
        
        # Turbulence intensity
        mean_velocity_mag = np.sqrt(mean_vx**2 + mean_vy**2)
        turbulence_intensity = np.sqrt(2 * np.mean(tke, axis=0) / 3) / (mean_velocity_mag + 1e-8)
        
        # Energy dissipation rate (proxy)
        velocity_gradients = np.gradient(velocity_x, axis=0)**2 + np.gradient(velocity_y, axis=0)**2
        dissipation_rate = np.mean(velocity_gradients, axis=0)
        
        return {
            'tke': tke,
            'tke_mean': np.mean(tke, axis=0),
            'reynolds_stress': {
                'uu': reynolds_uu,
                'vv': reynolds_vv,
                'uv': reynolds_uv
            },
            'turbulence_intensity': turbulence_intensity,
            'dissipation_rate': dissipation_rate,
            'velocity_x': velocity_x,
            'velocity_y': velocity_y
        }
    
    def compute_vorticity_risk(self, turbulence_metrics):
        """
        Compute vorticity-based risk measures
        """
        vx = turbulence_metrics['velocity_x']
        vy = turbulence_metrics['velocity_y']
        
        # Compute vorticity (curl of velocity field)
        dvx_dy = np.gradient(vx, axis=1)
        dvy_dx = np.gradient(vy, axis=0)
        vorticity = dvy_dx - dvx_dy
        
        # Vorticity-based risk metrics
        vorticity_magnitude = np.abs(vorticity)
        mean_vorticity = np.mean(vorticity, axis=0)
        vorticity_volatility = np.std(vorticity, axis=0)
        
        # Circulation strength (integral of vorticity)
        circulation = np.sum(vorticity, axis=0)
        
        # Vortex identification (Q-criterion)
        dvx_dx = np.gradient(vx, axis=0)
        dvy_dy = np.gradient(vy, axis=1)
        
        # Strain rate and rotation rate
        strain_rate = 0.5 * ((dvx_dx - dvy_dy)**2 + (dvx_dy + dvy_dx)**2)
        rotation_rate = 0.5 * (dvy_dx - dvx_dy)**2
        
        # Q-criterion: Q > 0 indicates vortex regions
        Q_criterion = 0.5 * (rotation_rate - strain_rate)
        
        return {
            'vorticity': vorticity,
            'vorticity_magnitude': vorticity_magnitude,
            'mean_vorticity': mean_vorticity,
            'vorticity_volatility': vorticity_volatility,
            'circulation': circulation,
            'Q_criterion': Q_criterion,
            'vortex_strength': np.mean(Q_criterion[Q_criterion > 0], axis=0) if np.any(Q_criterion > 0) else np.zeros(Q_criterion.shape[1])
        }
    
    def dynamic_position_sizing(self, turbulence_metrics, vorticity_metrics, base_position=1.0):
        """
        Dynamic position sizing based on fluid dynamics risk measures
        """
        # Risk scaling factors
        tke_factor = 1.0 / (1.0 + turbulence_metrics['tke_mean'])
        turbulence_factor = 1.0 / (1.0 + turbulence_metrics['turbulence_intensity'])
        vorticity_factor = 1.0 / (1.0 + np.abs(vorticity_metrics['mean_vorticity']))
        dissipation_factor = 1.0 / (1.0 + turbulence_metrics['dissipation_rate'])
        
        # Combined risk adjustment
        risk_weights = [0.3, 0.3, 0.2, 0.2]  # Weights for each factor
        combined_factor = (risk_weights[0] * tke_factor + 
                          risk_weights[1] * turbulence_factor +
                          risk_weights[2] * vorticity_factor +
                          risk_weights[3] * dissipation_factor)
        
        # Apply minimum and maximum position limits
        position_sizes = base_position * combined_factor
        position_sizes = np.clip(position_sizes, 0.1, 2.0)  # 10% minimum, 200% maximum
        
        return {
            'position_sizes': position_sizes,
            'tke_factor': tke_factor,
            'turbulence_factor': turbulence_factor,
            'vorticity_factor': vorticity_factor,
            'dissipation_factor': dissipation_factor,
            'combined_factor': combined_factor
        }

# Initialize risk manager
risk_manager = FluidRiskManager()

# Generate sample market data
market_data = risk_manager.generate_market_data(n_assets=5, n_periods=1000)

print("Sample market data generated:")
print(f"Assets: {market_data['returns'].shape[1]}")
print(f"Time periods: {market_data['returns'].shape[0]}")
print(f"Date range: {market_data['returns'].index[0]} to {market_data['returns'].index[-1]}")
print(f"Regimes: {market_data['regimes'].unique()}")

## Portfolio Flow Correlation and Risk Decomposition

### Flow-Based Portfolio Risk

Traditional portfolio risk: $\sigma_p^2 = w^T \Sigma w$

Fluid dynamics enhancement:
$$\sigma_{fluid}^2 = w^T (\Sigma + R_{turb} + \Lambda_{vort}) w$$

Where:
- $R_{turb}$ = turbulence-induced risk matrix
- $\Lambda_{vort}$ = vorticity correlation matrix

### Cross-Asset Flow Analysis

Examines how fluid patterns propagate across different assets in the portfolio.

In [ ]:
# Compute fluid dynamics risk metrics
turbulence_metrics = risk_manager.compute_turbulence_metrics(market_data['returns'])
vorticity_metrics = risk_manager.compute_vorticity_risk(turbulence_metrics)
position_sizing = risk_manager.dynamic_position_sizing(turbulence_metrics, vorticity_metrics)

# Advanced risk analysis
def compute_portfolio_fluid_risk(returns_df, weights, turbulence_metrics, vorticity_metrics):
    """
    Compute portfolio risk incorporating fluid dynamics effects
    """
    returns = returns_df.values
    n_assets = len(weights)
    
    # Traditional covariance matrix
    cov_matrix = np.cov(returns.T)
    
    # Turbulence-enhanced covariance
    tke_matrix = np.outer(turbulence_metrics['tke_mean'], turbulence_metrics['tke_mean'])
    turbulence_cov = cov_matrix * (1 + 0.5 * tke_matrix)
    
    # Vorticity correlation matrix
    vorticity_corr = np.corrcoef(vorticity_metrics['vorticity'].T)
    vorticity_risk = 0.1 * vorticity_corr * np.outer(np.abs(vorticity_metrics['mean_vorticity']), 
                                                     np.abs(vorticity_metrics['mean_vorticity']))
    
    # Combined fluid risk matrix
    fluid_cov = turbulence_cov + vorticity_risk
    
    # Portfolio risks
    traditional_risk = np.sqrt(weights.T @ cov_matrix @ weights)
    fluid_risk = np.sqrt(weights.T @ fluid_cov @ weights)
    
    return {
        'traditional_risk': traditional_risk,
        'fluid_risk': fluid_risk,
        'risk_enhancement': fluid_risk / traditional_risk,
        'cov_matrix': cov_matrix,
        'fluid_cov_matrix': fluid_cov,
        'turbulence_contribution': np.sqrt(weights.T @ turbulence_cov @ weights),
        'vorticity_contribution': np.sqrt(weights.T @ (cov_matrix + vorticity_risk) @ weights)
    }

def stress_test_fluid_scenarios(returns_df, weights, n_scenarios=1000):
    """
    Stress test portfolio under extreme fluid dynamics scenarios
    """
    returns = returns_df.values
    n_periods, n_assets = returns.shape
    
    scenarios = []
    
    for _ in range(n_scenarios):
        # Generate extreme fluid scenario
        scenario_type = np.random.choice(['turbulence_spike', 'vortex_formation', 'regime_shock', 'correlation_break'])
        
        if scenario_type == 'turbulence_spike':
            # Sudden increase in market turbulence
            volatility_multiplier = np.random.uniform(2, 5)
            scenario_returns = returns[-30:] * volatility_multiplier
        
        elif scenario_type == 'vortex_formation':
            # Trend reversal scenario
            reversal_strength = np.random.uniform(0.5, 2.0)
            scenario_returns = -returns[-30:] * reversal_strength
        
        elif scenario_type == 'regime_shock':
            # Sudden regime change
            shock_returns = np.random.normal(-0.05, 0.1, (30, n_assets))
            scenario_returns = shock_returns
        
        else:  # correlation_break
            # Correlation structure breaks down
            uncorr_returns = np.random.normal(0, np.std(returns, axis=0), (30, n_assets))
            scenario_returns = uncorr_returns
        
        # Compute scenario portfolio return
        portfolio_returns = scenario_returns @ weights
        cumulative_return = np.prod(1 + portfolio_returns) - 1
        max_drawdown = np.min(np.cumprod(1 + portfolio_returns)) - 1
        
        scenarios.append({
            'type': scenario_type,
            'cumulative_return': cumulative_return,
            'max_drawdown': max_drawdown,
            'volatility': np.std(portfolio_returns)
        })
    
    return pd.DataFrame(scenarios)

# Example portfolio weights (equal weight)
n_assets = market_data['returns'].shape[1]
equal_weights = np.ones(n_assets) / n_assets

# Risk-adjusted weights using fluid dynamics
risk_adjusted_weights = position_sizing['position_sizes'] / np.sum(position_sizing['position_sizes'])

# Compute portfolio risks
equal_weight_risk = compute_portfolio_fluid_risk(market_data['returns'], equal_weights, 
                                                turbulence_metrics, vorticity_metrics)
risk_adjusted_risk = compute_portfolio_fluid_risk(market_data['returns'], risk_adjusted_weights,
                                                 turbulence_metrics, vorticity_metrics)

# Stress testing
stress_results = stress_test_fluid_scenarios(market_data['returns'], equal_weights, n_scenarios=500)

print("Portfolio Risk Analysis:")
print(f"\nEqual Weight Portfolio:")
print(f"Traditional Risk: {equal_weight_risk['traditional_risk']:.4f}")
print(f"Fluid-Enhanced Risk: {equal_weight_risk['fluid_risk']:.4f}")
print(f"Risk Enhancement Factor: {equal_weight_risk['risk_enhancement']:.2f}x")

print(f"\nRisk-Adjusted Portfolio:")
print(f"Traditional Risk: {risk_adjusted_risk['traditional_risk']:.4f}")
print(f"Fluid-Enhanced Risk: {risk_adjusted_risk['fluid_risk']:.4f}")
print(f"Risk Enhancement Factor: {risk_adjusted_risk['risk_enhancement']:.2f}x")

print(f"\nStress Test Results:")
print(f"Worst case return: {stress_results['cumulative_return'].min():.2%}")
print(f"Maximum drawdown: {stress_results['max_drawdown'].min():.2%}")
print(f"95% VaR: {stress_results['cumulative_return'].quantile(0.05):.2%}")
print(f"Expected shortfall: {stress_results['cumulative_return'][stress_results['cumulative_return'] <= stress_results['cumulative_return'].quantile(0.05)].mean():.2%}")

## Risk-Adjusted Performance Metrics

### Fluid Sharpe Ratio

Enhanced Sharpe ratio incorporating turbulence effects:
$$Sharpe_{fluid} = \frac{\mu_p - r_f}{\sigma_{fluid}}$$

### Turbulence-Adjusted Returns

$$R_{adj} = R_{raw} \cdot (1 - \alpha \cdot TKE)$$

### Vorticity Risk Premium

Additional return required to compensate for vorticity-induced risk.

In [ ]:
# Create comprehensive risk management visualization
fig, axes = plt.subplots(3, 3, figsize=(18, 14))
fig.suptitle('Fluid Dynamics Risk Management Dashboard', fontsize=16, fontweight='bold')

# 1. Turbulent Kinetic Energy over time (top left)
ax1 = axes[0, 0]
tke_timeline = np.mean(turbulence_metrics['tke'], axis=1)
ax1.plot(market_data['returns'].index, tke_timeline, 'b-', alpha=0.7, linewidth=1.5)
ax1.fill_between(market_data['returns'].index, 0, tke_timeline, alpha=0.3)
ax1.set_title('Turbulent Kinetic Energy')
ax1.set_ylabel('TKE')
ax1.grid(True, alpha=0.3)

# Color by regime
regime_colors = {'normal': 'green', 'volatile': 'orange', 'trending': 'blue', 'crisis': 'red'}
for regime in regime_colors:
    mask = market_data['regimes'] == regime
    if mask.any():
        ax1.scatter(market_data['returns'].index[mask], tke_timeline[mask], 
                   c=regime_colors[regime], s=10, alpha=0.6, label=regime)
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# 2. Vorticity magnitude (top center)
ax2 = axes[0, 1]
vorticity_mag_timeline = np.mean(vorticity_metrics['vorticity_magnitude'], axis=1)
ax2.plot(market_data['returns'].index, vorticity_mag_timeline, 'r-', alpha=0.7, linewidth=1.5)
ax2.axhline(y=np.mean(vorticity_mag_timeline), color='black', linestyle='--', alpha=0.5, label='Mean')
ax2.set_title('Vorticity Magnitude')
ax2.set_ylabel('|Vorticity|')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Dynamic position sizing (top right)
ax3 = axes[0, 2]
asset_names = market_data['returns'].columns
x_pos = np.arange(len(asset_names))
bars = ax3.bar(x_pos, position_sizing['position_sizes'], alpha=0.7, 
               color=sns.color_palette("viridis", len(asset_names)))
ax3.axhline(y=1.0, color='red', linestyle='--', alpha=0.7, label='Base Position')
ax3.set_title('Dynamic Position Sizing')
ax3.set_xlabel('Assets')
ax3.set_ylabel('Position Size Multiplier')
ax3.set_xticks(x_pos)
ax3.set_xticklabels(asset_names, rotation=45)
ax3.legend()
ax3.grid(True, alpha=0.3)

# Add value labels on bars
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{height:.2f}', ha='center', va='bottom', fontsize=9)

# 4. Risk decomposition (middle left)
ax4 = axes[1, 0]
risk_components = ['Traditional', 'Turbulence', 'Vorticity', 'Combined']
risk_values = [
    equal_weight_risk['traditional_risk'],
    equal_weight_risk['turbulence_contribution'],
    equal_weight_risk['vorticity_contribution'],
    equal_weight_risk['fluid_risk']
]
colors = ['blue', 'orange', 'green', 'red']
bars = ax4.bar(risk_components, risk_values, color=colors, alpha=0.7)
ax4.set_title('Risk Decomposition')
ax4.set_ylabel('Portfolio Risk')
ax4.grid(True, alpha=0.3)

# Add value labels
for bar, value in zip(bars, risk_values):
    ax4.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.001,
             f'{value:.4f}', ha='center', va='bottom', fontsize=9)

# 5. Stress test results (middle center)
ax5 = axes[1, 1]
stress_results['cumulative_return'].hist(bins=30, alpha=0.7, ax=ax5, color='darkred')
ax5.axvline(x=stress_results['cumulative_return'].quantile(0.05), color='red', 
           linestyle='--', linewidth=2, label='5% VaR')
ax5.axvline(x=stress_results['cumulative_return'].mean(), color='blue', 
           linestyle='-', linewidth=2, label='Mean')
ax5.set_title('Stress Test Distribution')
ax5.set_xlabel('Cumulative Return')
ax5.set_ylabel('Frequency')
ax5.legend()
ax5.grid(True, alpha=0.3)

# 6. Correlation heatmap (middle right)
ax6 = axes[1, 2]
corr_matrix = np.corrcoef(vorticity_metrics['vorticity'].T)
im = ax6.imshow(corr_matrix, cmap='RdBu_r', vmin=-1, vmax=1)
ax6.set_title('Vorticity Correlation Matrix')
ax6.set_xticks(range(len(asset_names)))
ax6.set_yticks(range(len(asset_names)))
ax6.set_xticklabels(asset_names, rotation=45)
ax6.set_yticklabels(asset_names)
plt.colorbar(im, ax=ax6)

# Add correlation values
for i in range(len(asset_names)):
    for j in range(len(asset_names)):
        ax6.text(j, i, f'{corr_matrix[i, j]:.2f}', ha='center', va='center',
                color='white' if abs(corr_matrix[i, j]) > 0.5 else 'black', fontsize=8)

# 7. Regime analysis (bottom left)
ax7 = axes[2, 0]
regime_risk = market_data['regimes'].value_counts()
wedges, texts, autotexts = ax7.pie(regime_risk.values, labels=regime_risk.index, 
                                  autopct='%1.1f%%', startangle=90)
ax7.set_title('Market Regime Distribution')

# 8. Risk factors comparison (bottom center)
ax8 = axes[2, 1]
factors = ['TKE', 'Turbulence', 'Vorticity', 'Dissipation']
factor_values = [
    np.mean(position_sizing['tke_factor']),
    np.mean(position_sizing['turbulence_factor']),
    np.mean(position_sizing['vorticity_factor']),
    np.mean(position_sizing['dissipation_factor'])
]
bars = ax8.barh(factors, factor_values, color=['red', 'orange', 'green', 'blue'], alpha=0.7)
ax8.set_title('Risk Factor Contributions')
ax8.set_xlabel('Factor Value (lower = higher risk)')
ax8.grid(True, alpha=0.3)

# Add value labels
for i, (bar, value) in enumerate(zip(bars, factor_values)):
    ax8.text(value + 0.01, bar.get_y() + bar.get_height()/2.,
             f'{value:.3f}', ha='left', va='center', fontsize=9)

# 9. Performance summary (bottom right)
ax9 = axes[2, 2]
ax9.axis('off')

# Summary statistics
portfolio_returns = market_data['returns'] @ equal_weights
sharpe_ratio = portfolio_returns.mean() / portfolio_returns.std() * np.sqrt(252)
max_dd = (portfolio_returns.cumsum() - portfolio_returns.cumsum().expanding().max()).min()
calmar_ratio = portfolio_returns.mean() * 252 / abs(max_dd)

summary_text = f"""
RISK MANAGEMENT SUMMARY

Portfolio Metrics:
• Traditional Risk: {equal_weight_risk['traditional_risk']:.4f}
• Fluid-Enhanced Risk: {equal_weight_risk['fluid_risk']:.4f}
• Risk Enhancement: {equal_weight_risk['risk_enhancement']:.2f}x
• Sharpe Ratio: {sharpe_ratio:.3f}
• Calmar Ratio: {calmar_ratio:.3f}

Risk Factors:
• Mean TKE: {np.mean(turbulence_metrics['tke_mean']):.4f}
• Mean Turbulence: {np.mean(turbulence_metrics['turbulence_intensity']):.3f}
• Mean |Vorticity|: {np.mean(np.abs(vorticity_metrics['mean_vorticity'])):.4f}
• Dissipation Rate: {np.mean(turbulence_metrics['dissipation_rate']):.4f}

Stress Testing:
• 5% VaR: {stress_results['cumulative_return'].quantile(0.05):.2%}
• Expected Shortfall: {stress_results['cumulative_return'][stress_results['cumulative_return'] <= stress_results['cumulative_return'].quantile(0.05)].mean():.2%}
• Worst Case: {stress_results['cumulative_return'].min():.2%}
• Max Drawdown: {stress_results['max_drawdown'].min():.2%}

Position Sizing:
• Min Position: {position_sizing['position_sizes'].min():.2f}x
• Max Position: {position_sizing['position_sizes'].max():.2f}x
• Avg Position: {position_sizing['position_sizes'].mean():.2f}x

Status: {'🟢 LOW RISK' if equal_weight_risk['risk_enhancement'] < 1.2 else '🟡 MODERATE RISK' if equal_weight_risk['risk_enhancement'] < 1.5 else '🔴 HIGH RISK'}
"""

ax9.text(0.05, 0.95, summary_text, transform=ax9.transAxes, 
         fontsize=10, verticalalignment='top', fontfamily='monospace',
         bbox=dict(boxstyle='round,pad=0.5', facecolor='lightcyan', alpha=0.8))

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("FLUID DYNAMICS RISK MANAGEMENT ANALYSIS COMPLETE")
print("="*70)
print(f"Risk enhancement factor: {equal_weight_risk['risk_enhancement']:.2f}x traditional risk")
print(f"Dynamic position adjustments range: {position_sizing['position_sizes'].min():.2f}x to {position_sizing['position_sizes'].max():.2f}x")
print(f"Stress test scenarios evaluated: {len(stress_results)}")
print(f"Worst-case portfolio loss: {stress_results['cumulative_return'].min():.2%}")
print("\nKey Risk Management Features:")
print("• Turbulence-based risk metrics")
print("• Vorticity-driven position sizing")
print("• Multi-scenario stress testing")
print("• Real-time risk monitoring")
print("• Regime-aware risk models")
print("="*70)